# Full Registration
This notebook provides a comparison to the Fast-Global-Registration algorithm as it is implemented in the package open3d.

<b>NOTE</b>: The visualizations open a new window in your computer, in order to continue running, the visualization window have to be closed.

## Setup
Run the following cells to set up the notebook

In [1]:
import os
import sys
from copy import deepcopy

import numpy as np
import open3d as o3d
from ipywidgets import widgets

sys.path.append("..")
from demo.utils.fgr_o3d import run_fgr
from src.logic.normalization import normalize_point_clouds
from src.logic.registration import fast_global_registration
from demo.utils.visualizers import draw_registration_result
sys.path.pop()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


'..'

### Choose an Example
Before running the next cells, you can choose any of the examples provided in the dataset folder using the following dropdown selector.

In [2]:
dropdown = widgets.Dropdown(
    options=os.listdir('../dataset'),
    description='Example:',
    value="pairwise_no_noise_01_rot_05",
    layout={'width': 'max-content'},
    disabled=False,
)
dropdown

Dropdown(description='Example:', index=51, layout=Layout(width='max-content'), options=('kitchen', 'pairwise_n…

In [3]:
pcd1 = o3d.io.read_point_cloud(f"../dataset/{dropdown.value}/Depth_0000.ply")
pcd2 = o3d.io.read_point_cloud(f"../dataset/{dropdown.value}/Depth_0001.ply")

In [4]:
if not pcd1.normals or not pcd2.normals:
    pcd1.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd2.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

## Initial Alignment

In [5]:
draw_registration_result(source=pcd1, target=pcd2, transformation=np.identity(4))

## Results from the open3d Implementation

In [6]:
o3d_result = run_fgr(pcd1, pcd2)

In [7]:
draw_registration_result(source=pcd1, target=pcd2, transformation=o3d_result)

## Result from Our Implementation
Notice that our implementation is slower since it is written in python, which is known to be slower than c++. The code uses loops only in the iterative optimization, but it uses the least-squares implementation from scipy, which is also known to be rather slow.

In [ ]:
pcd_p, pcd_q = normalize_point_clouds(deepcopy(pcd1), deepcopy(pcd2))
our_result = fast_global_registration(pcd_p, pcd_q)

In [ ]:
draw_registration_result(source=pcd_q, target=pcd_p, transformation=our_result)